In [1]:
import numpy as np
import matplotlib.pyplot as plt
from oracle import Oracle
from accuracy_test import accuracy_check

In [2]:
def decentralizedSGD(x0, lr, max_iter, W, oracle):
    
    x_t = x0
    
    x_store = np.zeros((max_iter, x0.shape[0], x0.shape[1]))
    
    for t in range(max_iter):
         
        noise = np.random.normal(0, 1, size=x_t.shape)
        
        f, df = oracle(x_t + noise)
        
        y_t = x_t - lr * df
        x_t = y_t @ W
        
        x_store[t, :, :] = x_t
        
    return x_store

def MetropolisHastings(W):
    
    degrees = np.sum(W, axis=1)
    
    for i in range(W.shape[0]):
        for j in range(i, W.shape[0]):
            
            if W[i, j] != 0:
                
                weight = min(1 / (degrees[i] + 1), 1 / (degrees[j] + 1))
                
                W[i, j] = weight
                W[j, i] = weight
                
    return W


def toMatrix(n_nodes, adj_list):
    
    matrix = np.zeros((n_nodes, n_nodes))
    
    for node in adj_list:
        
        neighbors = adj_list[node]
        
        for neighbor in neighbors:
            
            matrix[node, neighbor] = 1
            matrix[neighbor, node] = 1
            
    return matrix.astype(np.float64)


def buildTopology(n_nodes, topology):
    
    if topology == "dense":
        
        W = np.ones((n_nodes, n_nodes))
        
        for i in range(n_nodes):
            W[i, i] = 0
            
        return MetropolisHastings(W.astype(np.float64))
    
    elif topology == "ring":
        
        connections = {} # adjacency list
        
        for node in range(n_nodes):
            
            if node == n_nodes - 1:
                connections[node] = [0]
            else:
                connections[node] = [node + 1]
        
        W = toMatrix(n_nodes, connections)
        
        return MetropolisHastings(W)
    
    elif topology == "centralized":
        
        connections = {} # adjacency list
        
        for node in range(1, n_nodes):
            connections[node] = [0] # node 0 is central node
            
        W = toMatrix(n_nodes, connections)
        
        return MetropolisHastings(W)
    
    else:
        print("Wrong topology")
        

def experiment(list_nodes, n_params, topology, func_type, max_iter, lr, error_threshold, plot=True):
    
    results = []
    
    for n_nodes in list_nodes:
        
        print("Starting experiment on {} nodes.".format(n_nodes))
        
        temp = []
        
        for _ in range(5): # Do average
            
            x0 = np.ones((n_params, n_nodes)).astype(np.float64) #np.random.randn(n_params, n_nodes)
            W = buildTopology(n_nodes, topology)
            
            oracle = Oracle(func_type, n_params, n_nodes)

            x_store = decentralizedSGD(x0, lr, max_iter, W, oracle)

            x_star, f_star = oracle.getMin()

            mu = oracle.getMu()

            errors = accuracy_check(func_type, x_store, oracle, x_star, f_star, mu, W, plot)
            iter_below_thresh = [iter_ + 1 for iter_, error in enumerate(errors) if error < error_threshold]

            if len(iter_below_thresh) > 0:
                temp.append(iter_below_thresh[0])
            else:
                print("Didn't converge below threshold in {} iterations".format(max_iter))
        
        mean = np.mean(temp)
        print("Converged within {} in {} iterations on average.".format(error_threshold, mean))
        print()
        
        results.append(mean)
    
    return results
    

def plotIter(num_nodes, iters):
    
    plt.plot(num_nodes, iters)
    plt.xlabel("Number of nodes")
    plt.ylabel("Number of iterations")
    plt.show()

In [5]:
n_params = 10
topology = "ring"
func_type = "convex"
max_iter = 10000
lr = 1e-2
threshold = 0.1

num_nodes = list(range(2, 200, 20))

results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, False)

plotIter(num_nodes, results)

Starting experiment on 2 nodes.
0.0347890682351442
0.05123640833210436
0.050100986855312585
0.05061219887037851
0.05043724388680036
Converged within 0.1 in 4745.4 iterations on average.

Starting experiment on 22 nodes.
0.038280416339730526
0.052541461404040236
0.02543067975551952
0.041501722818089
0.04572846498590851
Converged within 0.1 in 4070.6 iterations on average.

Starting experiment on 42 nodes.


KeyboardInterrupt: 